### Tight Integration

In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'1.1.0'

In [3]:
from tensorflow.contrib import keras

---

### Tensorboard Integration

In [1]:
from keras.datasets import cifar100

(X_train, Y_train), (X_test, Y_test) = cifar100.load_data(label_mode='fine')

Using TensorFlow backend.


In [2]:
from keras import backend as K

img_rows, img_cols = 32, 32

if K.image_data_format() == 'channels_first':
    shape_ord = (3, img_rows, img_cols)
else:  # channel_last
    shape_ord = (img_rows, img_cols, 3)

In [3]:
shape_ord

(32, 32, 3)

In [4]:
X_train.shape

(50000, 32, 32, 3)

In [5]:
import numpy as np
nb_classes = len(np.unique(Y_train))

In [6]:
from keras.applications import vgg16
from keras.layers import Input

In [7]:
vgg16_model = vgg16.VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape_ord))
vgg16_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
__________

In [8]:
for layer in vgg16_model.layers:
    layer.trainable = False  # freeze layer

In [9]:
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization

In [10]:
x = Flatten(input_shape=vgg16_model.output.shape)(vgg16_model.output)
x = Dense(4096, activation='relu', name='ft_fc1')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
predictions = Dense(nb_classes, activation = 'softmax')(x)

In [11]:
from keras.models import Model

In [12]:
#create graph of your new model
model = Model(inputs=vgg16_model.input, outputs=predictions)

#compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
__________

### `TensorBoard` Callback

In [14]:
from keras.callbacks import TensorBoard

```python

# Arguments
    log_dir: the path of the directory where to save the log
        files to be parsed by TensorBoard.
    histogram_freq: frequency (in epochs) at which to compute activation
        and weight histograms for the layers of the model. If set to 0,
        histograms won't be computed. Validation data (or split) must be
        specified for histogram visualizations.
    write_graph: whether to visualize the graph in TensorBoard.
        The log file can become quite large when
        write_graph is set to True.
    write_grads: whether to visualize gradient histograms in TensorBoard.
        `histogram_freq` must be greater than 0.
    write_images: whether to write model weights to visualize as
        image in TensorBoard.
    embeddings_freq: frequency (in epochs) at which selected embedding
        layers will be saved.
    embeddings_layer_names: a list of names of layers to keep eye on. If
        None or empty list all the embedding layer will be watched.
    embeddings_metadata: a dictionary which maps layer name to a file name
        in which metadata for this embedding layer is saved. 
```

See the [details](https://www.tensorflow.org/how_tos/embedding_viz/#metadata_optional)
about metadata files format. In case if the same metadata file is used for all embedding layers, string can be passed.

In [15]:
## one-hot Encoding of labels (1 to 100 classes)
from keras.utils import np_utils
Y_train.shape

(50000, 1)

In [16]:
Y_train = np_utils.to_categorical(Y_train)

In [17]:
Y_train.shape

(50000, 100)

In [19]:
def generate_batches(X, Y, batch_size=128):
    """"""
    # Iterations has to go indefinitely
    start = 0
    while True:
        yield (X[start:start+batch_size], Y[start:start+batch_size])
        start=batch_size

batch_size = 64
steps_per_epoch = np.floor(X_train.shape[0] / batch_size)
model.fit_generator(generate_batches(X_train, Y_train, batch_size=batch_size),
                    steps_per_epoch=steps_per_epoch, epochs=20, verbose=1, 
                    callbacks=[TensorBoard(log_dir='./logs', histogram_freq=10, 
                                           write_graph=True, write_images=False, 
                                           embeddings_freq=10, 
                                           embeddings_layer_names=['block1_conv2', 'block5_conv1', 'ft_fc1'], 
                                           embeddings_metadata=None)])

INFO:tensorflow:Summary name block1_conv1/kernel:0 is illegal; using block1_conv1/kernel_0 instead.
INFO:tensorflow:Summary name block1_conv1/bias:0 is illegal; using block1_conv1/bias_0 instead.
INFO:tensorflow:Summary name block1_conv2/kernel:0 is illegal; using block1_conv2/kernel_0 instead.
INFO:tensorflow:Summary name block1_conv2/bias:0 is illegal; using block1_conv2/bias_0 instead.
INFO:tensorflow:Summary name block2_conv1/kernel:0 is illegal; using block2_conv1/kernel_0 instead.
INFO:tensorflow:Summary name block2_conv1/bias:0 is illegal; using block2_conv1/bias_0 instead.
INFO:tensorflow:Summary name block2_conv2/kernel:0 is illegal; using block2_conv2/kernel_0 instead.
INFO:tensorflow:Summary name block2_conv2/bias:0 is illegal; using block2_conv2/bias_0 instead.
INFO:tensorflow:Summary name block3_conv1/kernel:0 is illegal; using block3_conv1/kernel_0 instead.
INFO:tensorflow:Summary name block3_conv1/bias:0 is illegal; using block3_conv1/bias_0 instead.
INFO:tensorflow:Summ

#### Runing Tensorboard

In [22]:
%%bash
python -m tensorflow.tensorboard --logdir=./logs

Starting TensorBoard b'47' at http://0.0.0.0:6006
(Press CTRL+C to quit)


bash: line 1: 87778 Killed                  python -m tensorflow.tensorboard --logdir=./logs
